In [1]:
import requests
from bs4 import BeautifulSoup
import json

In [3]:
url = "https://www.hireitpeople.com/resume-database/66-business-analyst-resumes/627226-business-analyst-tester-resume-new-haven-ct"
response = requests.get(url)
soup = BeautifulSoup(response.text, "lxml")

In [4]:
data = {}
target_divs = soup.find_all("div", class_=["media-body", "single-post-body"])

for div in target_divs:
    for u_tag in div.find_all("u"):
        key = u_tag.get_text(strip=True)

        if key == "PROFESSIONAL EXPERIENCE":
            experiences = []

            p_tags = u_tag.find_all_next("p")
            strong_p_tags = [p for p in p_tags if p.find("strong")]

            i = 0
            while i < len(strong_p_tags):
                exp_data = {}

                if i < len(strong_p_tags):    # company_name
                    exp_data["company_name"] = strong_p_tags[i].get_text(strip=True)
                    i += 1

                if i < len(strong_p_tags):    # job_role
                    exp_data["job_role"] = strong_p_tags[i].get_text(strip=True)
                    i += 1

                if i < len(strong_p_tags):    # plain text in them
                    key3 = strong_p_tags[i].find("strong").get_text(strip=True)
                    val3 = strong_p_tags[i].get_text(strip=True).replace(key3, "").strip()
                    exp_data[key3] = val3
                    i += 1

                if i < len(strong_p_tags):
                    key4 = strong_p_tags[i].find("strong").get_text(strip=True)
                    ul_tag = strong_p_tags[i].find_next("ul")
                    if ul_tag:
                        val4 = [li.get_text(strip=True) for li in ul_tag.find_all("li")]
                    else:
                        val4 = []
                    exp_data[key4] = val4
                    i += 1

                experiences.append(exp_data)

            data[key] = experiences

        elif key == "TECHNICAL SKILLS":
            tech_skills = {}
            for sibling in u_tag.find_all_next("p"):
                if sibling.find("u"):
                    break
                strong = sibling.find("strong")
                if strong:
                    sub_key = strong.get_text(strip=True).rstrip(":")
                    sub_val = sibling.get_text(" ", strip=True).replace(strong.get_text(strip=True), "").strip()
                    if sub_val:
                        tech_skills[sub_key] = sub_val
            data[key] = tech_skills

        else:
            ul_tag = u_tag.find_next("ul")
            if ul_tag:
                values = [li.get_text(strip=True) for li in ul_tag.find_all("li")]
            else:
                values = []
            data[key] = values


In [6]:
json_output = json.dumps(data, indent=4, ensure_ascii=False)
print(json_output)

with open("scraped_data.json", "w", encoding="utf-8") as f:
    f.write(json_output)